Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

#### ~~Tweet_id is encoded using approximate hashing which caused hashing collisions. Exact encoding of tweet_id is blocked by a bug in cudf. All other columns are using exact encoding.~~ 
Fixed

In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
start = time.time()
very_start = time.time()

In [2]:
#import pandas as pd, 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import cudf, cupy, time, rmm
cudf.__version__

'0.16.0'

In [3]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [4]:
'''cluster = LocalCUDACluster(ip='10.2.61.36',protocol="ucx", 
                           rmm_pool_size="31GB",
                           enable_tcp_over_ucx=True, enable_nvlink=True)
#cluster = LocalCUDACluster()
client = Client(cluster)
client'''

'cluster = LocalCUDACluster(ip=\'10.2.61.36\',protocol="ucx", \n                           rmm_pool_size="31GB",\n                           enable_tcp_over_ucx=True, enable_nvlink=True)\n#cluster = LocalCUDACluster()\nclient = Client(cluster)\nclient'

In [5]:
%%time

NP = 4 #16
#path = './'#'/home/pinto/rapid_data/recsys'

df = dask_cudf.read_csv('/home/pinto/training_s.tsv', sep='\x01', header=None)#, dtype=DTYPES)
df = df.repartition(npartitions=NP)
df, = dask.persist(df)
#_ = wait(df)
print('number of rows:',len(df))

number of rows: 1346935
CPU times: user 2.64 s, sys: 940 ms, total: 3.58 s
Wall time: 3.58 s


In [6]:
%%time
features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]
df.columns = features

df = df.drop('text_tokens', axis=1)
df, = dask.persist(df)
#_ = wait(df)
df.head()

CPU times: user 160 ms, sys: 16 ms, total: 176 ms
Wall time: 172 ms


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like
0,<NA>,D4D1EBDE74F74C5DA529959AF979625C,<NA>,<NA>,<NA>,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,...,0000006C3074607050F1339DDCB890BB,27428,600,False,1520948869,False,<NA>,<NA>,<NA>,<NA>
1,<NA>,BFB529DAB6D384EB83E899A72AB3830D,Photo,<NA>,<NA>,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,39786,...,000005C520010F8917EEAB6F5B6EC1C4,17,77,False,1569692352,True,<NA>,<NA>,<NA>,<NA>
2,<NA>,519078C7834E9642508F72A6C2D0F3B7,<NA>,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,141726,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,<NA>,<NA>,<NA>,1.581247087e+09
3,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,<NA>,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,3044,...,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,<NA>,<NA>,<NA>,<NA>
4,<NA>,89C1298C55EB3D68E2784F0BFB69E6F8,<NA>,<NA>,<NA>,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,141370,...,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,<NA>,<NA>,<NA>,<NA>


In [7]:
df.dtypes

hashtags               object
tweet_id               object
media                  object
links                  object
domains                object
tweet_type             object
language               object
timestamp               int64
a_user_id              object
a_follower_count        int64
a_following_count       int64
a_is_verified            bool
a_account_creation      int64
b_user_id              object
b_follower_count        int64
b_following_count       int64
b_is_verified            bool
b_account_creation      int64
b_follows_a              bool
reply                 float64
retweet               float64
retweet_comment       float64
like                  float64
dtype: object

In [8]:
%%time
df['id']   = 1
df['id']   = df['id'].cumsum()
df['id'] = df['id'].astype('int32')

df['reply']   = df['reply'].fillna(0)
df['retweet'] = df['retweet'].fillna(0)
df['retweet_comment'] = df['retweet_comment'].fillna(0)
df['like']    = df['like'].fillna(0)

df['reply']   = df['reply'].astype('int32')
df['retweet'] = df['retweet'].astype('int32')
df['retweet_comment'] = df['retweet_comment'].astype('int32')
df['like']    = df['like'].astype('int32')
df, = dask.persist(df)
#_ = wait(df)

CPU times: user 976 ms, sys: 300 ms, total: 1.28 s
Wall time: 1.27 s


In [9]:
%%time

df['timestamp']         = df['timestamp'].astype( np.int32 )
df['a_follower_count']  = df['a_follower_count'].astype( np.int32 )
df['a_following_count'] = df['a_following_count'].astype( np.int32 )
df['a_account_creation']= df['a_account_creation'].astype( np.int32 )
df['b_follower_count']  = df['b_follower_count'].astype( np.int32 )
df['b_following_count'] = df['b_following_count'].astype( np.int32 )
df['b_account_creation']= df['b_account_creation'].astype( np.int32 )

df, = dask.persist(df)
#_ = wait(df)
df.head()

CPU times: user 332 ms, sys: 216 ms, total: 548 ms
Wall time: 545 ms


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id
0,<NA>,D4D1EBDE74F74C5DA529959AF979625C,<NA>,<NA>,<NA>,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,...,27428,600,False,1520948869,False,0,0,0,0,1
1,<NA>,BFB529DAB6D384EB83E899A72AB3830D,Photo,<NA>,<NA>,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,39786,...,17,77,False,1569692352,True,0,0,0,0,2
2,<NA>,519078C7834E9642508F72A6C2D0F3B7,<NA>,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,141726,...,2,29,False,1568107028,False,0,0,0,1581247087,3
3,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,<NA>,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,3044,...,3,30,False,1568107028,False,0,0,0,0,4
4,<NA>,89C1298C55EB3D68E2784F0BFB69E6F8,<NA>,<NA>,<NA>,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,141370,...,3,30,False,1568107028,False,0,0,0,0,5


In [10]:
df.dtypes

hashtags              object
tweet_id              object
media                 object
links                 object
domains               object
tweet_type            object
language              object
timestamp              int32
a_user_id             object
a_follower_count       int32
a_following_count      int32
a_is_verified           bool
a_account_creation     int32
b_user_id             object
b_follower_count       int32
b_following_count      int32
b_is_verified           bool
b_account_creation     int32
b_follows_a             bool
reply                  int32
retweet                int32
retweet_comment        int32
like                   int32
id                     int32
dtype: object

In [11]:
%%time
dv = dask_cudf.read_csv('/home/pinto/val.tsv', sep='\x01', header=None)
dv = dv.repartition(npartitions=NP)
dv, = dask.persist(dv)
#_ = wait(dv)
print('number of rows:',len(dv))

number of rows: 9760684
CPU times: user 3.84 s, sys: 1.75 s, total: 5.59 s
Wall time: 5.59 s


In [12]:
%%time
print(dv.head())
features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]
dv.columns = features
dv = dv.drop('text_tokens', axis=1)
dv, = dask.persist(dv)
#_ = wait(dv)

                                                   0     1  \
0  101\t10117\t140\t119\t142\t119\t152\t119\t1010...  <NA>   
1  101\t10105\t10817\t10124\t59232\t18121\t15629\...  <NA>   
2  101\t48561\t10116\t67737\t18554\t36371\t10989\...  <NA>   
3  101\t100055\t69940\t10414\t159\t11305\t11166\t...  <NA>   
4  101\t62154\t32221\t71843\t10143\t10237\t15507\...  <NA>   

                                  2      3                                 4  \
0  373C0F43762B7CEC1D75728BE8A33891   <NA>  A2CE3A1941BA410A1C31496C355EFCD7   
1  773A92D9E4824D06105C02BD044BB20A   <NA>                              <NA>   
2  218A6C27871801759F7380D7C41694A6   <NA>  5C683B5A29B308CADD0D7EFA7C9C32D3   
3  AB817EBA68064A0C8CBF4A6C059D92DC  Photo  E925556EE312213AD98C4D9F131D7A8D   
4  349120C1E2801857530393F16D4653A5   <NA>                              <NA>   

                                  5         6  \
0  E14AF8A8D257BB47587843FE7D08382B  TopLevel   
1                              <NA>     Quote   

In [13]:
%%time

dv['reply']           = 0
dv['retweet']         = 0
dv['retweet_comment'] = 0
dv['like']            = 0

dv['id']   = 1
dv['id']   = dv['id'].cumsum()
dv['id'] = dv['id'] + len(df)
dv['id'] = dv['id'].astype('int32')

dv['reply']           = dv['reply'].astype( np.int32 )
dv['retweet']         = dv['retweet'].astype( np.int32 )
dv['retweet_comment'] = dv['retweet_comment'].astype( np.int32 )
dv['like']            = dv['like'].astype( np.int32 )

dv['timestamp']         = dv['timestamp'].astype( np.int32 )
dv['a_follower_count']  = dv['a_follower_count'].astype( np.int32 )
dv['a_following_count'] = dv['a_following_count'].astype( np.int32 )
dv['a_account_creation']= dv['a_account_creation'].astype( np.int32 )
dv['b_follower_count']  = dv['b_follower_count'].astype( np.int32 )
dv['b_following_count'] = dv['b_following_count'].astype( np.int32 )
dv['b_account_creation']= dv['b_account_creation'].astype( np.int32 )
dv, = dask.persist(dv)
#_ = wait(dv)

CPU times: user 1.95 s, sys: 2.27 s, total: 4.22 s
Wall time: 4.22 s


In [14]:
%%time
dt = dask_cudf.read_csv(f'/home/pinto/test.tsv', sep='\x01', header=None)
dt = dt.repartition(npartitions=NP)
dt, = dask.persist(dt)
#_ = wait(dt)

CPU times: user 3.84 s, sys: 1.76 s, total: 5.6 s
Wall time: 5.61 s


In [15]:
%%time
print(dt.head())
features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]
dt.columns = features
dt = dt.drop('text_tokens', axis=1)
dt, = dask.persist(dt)
#_ = wait(dt)
print('number of rows:',len(dt))

                                                   0  \
0  101\t3100\t5477\t3028\t4348\t1924\t111806\t186...   
1  101\t56898\t137\t36110\t10400\t168\t64062\t131...   
2  101\t10117\t23672\t12610\t86532\t11205\t90138\...   
3  101\t13690\t14372\t119\t119\t19281\t25444\t984...   
4  101\t56898\t137\t171\t64791\t168\t11499\t10330...   

                                                   1  \
0                                               <NA>   
1  024FE90EC2C01B3CDC46A5A90D66B020\t1B78BDD9C7FF...   
2                                               <NA>   
3                                               <NA>   
4  9533703A3BB1CBA49AC0547C4A9F8043\t4C57C1D9063D...   

                                  2      3     4     5         6  \
0  04746004AA1F5498834CE7A4C6343D1A   <NA>  <NA>  <NA>  TopLevel   
1  B5C4CBE185831F3E5A58A4D81118D4C7   <NA>  <NA>  <NA>   Retweet   
2  8747BA89E3245FE4097C9E4E0AE40862  Video  <NA>  <NA>  TopLevel   
3  1CD2139E249349CBD5A0576D163C5EF8  Photo  <NA>  <NA>

In [16]:
%%time
dt['reply']           = 0
dt['retweet']         = 0
dt['retweet_comment'] = 0
dt['like']            = 0

dt['id']   = 1
dt['id']   = dt['id'].cumsum()
dt['id']   = dt['id']+len(df)+len(dv)
dt['id']   = dt['id'].astype('int32')

dt['reply']           = dt['reply'].astype( np.int32 )
dt['retweet']         = dt['retweet'].astype( np.int32 )
dt['retweet_comment'] = dt['retweet_comment'].astype( np.int32 )
dt['like']            = dt['like'].astype( np.int32 )

dt['timestamp']         = dt['timestamp'].astype( np.int32 )
dt['a_follower_count']  = dt['a_follower_count'].astype( np.int32 )
dt['a_following_count'] = dt['a_following_count'].astype( np.int32 )
dt['a_account_creation']= dt['a_account_creation'].astype( np.int32 )
dt['b_follower_count']  = dt['b_follower_count'].astype( np.int32 )
dt['b_following_count'] = dt['b_following_count'].astype( np.int32 )
dt['b_account_creation']= dt['b_account_creation'].astype( np.int32 )

dt, = dask.persist(dt)
#_ = wait(dt)
print(df.shape,dv.shape,dt.shape)
dt.head()

(Delayed('int-bbe2c33e-1fa5-4961-b03d-7fe02f30d81e'), 24) (Delayed('int-0e26b3ba-2380-4dfb-bace-87f3ce12ed31'), 24) (Delayed('int-5def1a2f-5d8e-4d1c-a203-0a802aae85e7'), 24)
CPU times: user 2.02 s, sys: 2.3 s, total: 4.32 s
Wall time: 4.32 s


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id
0,<NA>,04746004AA1F5498834CE7A4C6343D1A,<NA>,<NA>,<NA>,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581759640,6720CC7830F94CB7465CA283300DB010,119,...,111,673,False,1478011810,True,0,0,0,0,11107620
1,024FE90EC2C01B3CDC46A5A90D66B020\t1B78BDD9C7FF...,B5C4CBE185831F3E5A58A4D81118D4C7,<NA>,<NA>,<NA>,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581668217,7DDC67265CFB6E0B4820E0BD0E33A8D3,189,...,111,673,False,1478011810,True,0,0,0,0,11107621
2,<NA>,8747BA89E3245FE4097C9E4E0AE40862,Video,<NA>,<NA>,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581876000,1C0B9850DE2A34BA8B8DF413F5C13233,405907,...,15,123,False,1385502405,False,0,0,0,0,11107622
3,<NA>,1CD2139E249349CBD5A0576D163C5EF8,Photo,<NA>,<NA>,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581646803,94FBB99CE36D7A152DABA57CEF62F75E,111841,...,141,955,False,1335110299,False,0,0,0,0,11107623
4,9533703A3BB1CBA49AC0547C4A9F8043\t4C57C1D9063D...,BDFA89609AC3EDDE250DCECD964B5890,Photo,<NA>,<NA>,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581623428,311BB1E051070C2639BC632ECDE69C43,3577,...,139,953,False,1335110299,False,0,0,0,0,11107624


In [17]:
train_size = len(df)#.shape[0]
test0_size = len(dv)#.shape[0]
test1_size = len(dt)#.shape[0]
print(train_size,test0_size,test1_size)

1346935 9760684 9765321


In [18]:
%%time
df = dask_cudf.concat( [df,dv,dt] )
df, = dask.persist(df)
#wait(df)
del dv, dt

CPU times: user 104 ms, sys: 4 ms, total: 108 ms
Wall time: 106 ms


In [19]:
df.head()['language']

0    22C448FF81263D4BAF2A176145EE9EAD
1    B9175601E87101A984A50F8A62A1C374
2    22C448FF81263D4BAF2A176145EE9EAD
3    22C448FF81263D4BAF2A176145EE9EAD
4    22C448FF81263D4BAF2A176145EE9EAD
Name: language, dtype: object

In [20]:
df.info()

<class 'dask_cudf.core.DataFrame'>
Columns: 24 entries, hashtags to id
dtypes: object(9), bool(3), int32(12)

In [21]:
print(df.npartitions,len(df))

12 20872940


In [22]:
"""
%%time
df['tweet_id'] = df['tweet_id'].map_partitions(lambda cudf:cudf.hash_encode(stop=1_000_000_000))
df['tweet_id'] = df['tweet_id'].astype( np.int32 )
#df['tweet_id'] = df['tweet_id'].map_partitions(lambda cudf:cudf.hash_values()%1_000_000_000)
df, = dask.persist(df)
_ = wait(df)
df.head()
"""

"\n%%time\ndf['tweet_id'] = df['tweet_id'].map_partitions(lambda cudf:cudf.hash_encode(stop=1_000_000_000))\ndf['tweet_id'] = df['tweet_id'].astype( np.int32 )\n#df['tweet_id'] = df['tweet_id'].map_partitions(lambda cudf:cudf.hash_values()%1_000_000_000)\ndf, = dask.persist(df)\n_ = wait(df)\ndf.head()\n"

In [23]:
%%time
df['media'] = df['media'].fillna( '' )
def split_join(ds,sep):
    df = ds.str.split(sep)
    df[0] = df[0].fillna('')
    df[1] = df[1].fillna('')
    res = df[0]+'_'+df[1]
    del df
    return res

df['media'] = df['media'].map_partitions( lambda x:  split_join(x,'\t'), meta=('O'))

df, = dask.persist(df)
#_ = wait(df)

CPU times: user 720 ms, sys: 640 ms, total: 1.36 s
Wall time: 1.36 s


In [24]:
def factorize_small_cardinality(df,col):
    tmp_col = f'{col}_encode'
    tmp = df[col].unique().compute()
    tmp = tmp.to_frame().reset_index()
    tmp = tmp.rename(columns={'index':tmp_col})
    df = df.merge(tmp,on=col,how='left')
    df, = dask.persist(df)
    #wait(df)
    head=df.head()
    del tmp
    df = df.drop(col,axis=1)
    df, = dask.persist(df)
    #wait(df)
    df.columns = [i if i!=tmp_col else col for i in df.columns ]
    return df,head

In [25]:
%%time
for col in ['language','tweet_type','media']:
    df,_ = factorize_small_cardinality(df,col)
    df[col] = df[col].astype('int8')

CPU times: user 3.15 s, sys: 2.36 s, total: 5.51 s
Wall time: 5.52 s


In [26]:
df.head()

,hashtags,tweet_id,links,domains,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,language,tweet_type,media
0,<NA>,CD330B5F269D018305E932FB850C344E,<NA>,<NA>,1580954223,C406A6A841810BE02BC2111EED1DC1A4,31520,14265,False,1464697339,...,1418858585,True,0,0,0,1580962767,17441,11,2,4
1,<NA>,F731F6E9867E0A6027C98B918DF97AAC,<NA>,<NA>,1580971774,B6779812B89BD1CFD199E8AEAD0B54D5,603,992,False,1420151666,...,1545552854,True,0,0,0,0,17442,3,1,12
2,AB2A66C6CED66073399114767974ECED,46B37C1036C68BB82694620BF270D474,<NA>,<NA>,1581112817,C0E2578644FF152C4B55C34DC3E38420,220957,1067,False,1296698336,...,1490827297,False,0,0,0,0,17443,3,2,4
3,145A79B89065771A2D791917E793B2FE,1163132E718D8BEBE5B14B8E73AFC4E9,<NA>,<NA>,1581436665,9170F5E5DF35DFE54C14BE568F8591A7,63052,2,True,1495465681,...,1421321918,False,0,0,0,1581504659,17444,54,2,4
4,<NA>,16E38DDAAD820D2EDC097B1B656534D7,<NA>,<NA>,1581434663,F8D56C50A02A399D8DDAA87B69BD35B8,172719,75,True,1506859455,...,1421321918,False,0,0,0,1581487190,17445,63,2,4


In [27]:
%%time
tweet = df[['tweet_id']]
tweet = tweet.drop_duplicates(split_out=16)
tweet['tweet_encode'] = 1
tweet['tweet_encode'] = tweet['tweet_encode'].cumsum()
tweet, = dask.persist(tweet)
#_ = wait(tweet)
tweet.head()

CPU times: user 2.36 s, sys: 1.26 s, total: 3.62 s
Wall time: 3.63 s


,tweet_id,tweet_encode
1570047,00001F0134EBC75A77354B68CA735691,1
1389577,00001FFF863F5454B282601E5065B13D,2
307361,00004198C6D86783B29ED850192BCE1A,3
28401,00004564C2B60DB475C5B8E3F4EBD692,4
1393737,00005084E7058BA1496FA276088C24E0,5


In [28]:
%%time
df = df.merge(tweet,on='tweet_id',how='left')
df = df.drop('tweet_id',axis=1)
df.columns = [i if i!='tweet_encode' else 'tweet_id' for i in df.columns]
df, = dask.persist(df)
#wait(df)
del tweet
df.head()

CPU times: user 2.55 s, sys: 1.56 s, total: 4.1 s
Wall time: 4.11 s


,hashtags,links,domains,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,...,b_follows_a,reply,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id
0,5F2EB9FF00146CA53D8A774B6221D7B7,<NA>,<NA>,1581251156,4C2E8E24D98EF6FC978B2BD2900228FD,912322,574,False,1277893699,0133CCC28155810BA550CB9C658C91CF,...,False,0,0,0,0,86980,11,1,12,198424
1,B26810959DE7C1207F8819E456F07959,<NA>,<NA>,1581211034,19A1E782CEB1923C73F1BCD7EF0B8F09,286087,1080,True,1229127674,01E8029AAF5B2F9353CF6F83668CF3B4,...,False,0,0,0,0,137161,54,1,12,369094
2,<NA>,<NA>,<NA>,1581012891,D53FAF16AD2B05892B15112D138F6C87,140753,343,False,1315622198,018C49A4E371B922131A2C7DE5E4F400,...,False,0,0,0,1581105743,111619,3,2,12,337707
3,<NA>,0B50B15EA016F6BCDA59AE3A5E62AD6D,3AB8559C033305E17797F557488967C3,1580999813,AAC58C42CD47D9E188D5396768A42EE7,87727,2999,True,1258441967,0147D476C81B8C498920B97052738177,...,False,0,0,0,1581027706,92473,11,2,12,235223
4,<NA>,<NA>,<NA>,1581006647,3F73042B6A3069E99AB67A79586042AA,250354,272,True,1438657976,01AF75E88D5A6356E6E7A012AFA10A9D,...,False,0,0,0,0,121578,11,2,4,715906


In [29]:
#%%time
#df = df.repartition(npartitions=1024)
#df, = dask.persist(df)
#_ = wait(df)

In [30]:
%%time
user_a = df[['a_user_id']].drop_duplicates(split_out=16)
user_a, = dask.persist(user_a)
#_ = wait(user_a)
user_b = df[['b_user_id']].drop_duplicates(split_out=16)
user_b, = dask.persist(user_b)
#wait(user_b)
print(len(user_a),len(user_b),len(df))

user_a.columns = ['user_id']
user_b.columns = ['user_id']
user_b['dummy'] = 1
user_a = user_a.merge(user_b,on='user_id',how='outer')
user_a = user_a.drop('dummy',axis=1)
user_a, = dask.persist(user_a)
#wait(user_a)
print(len(user_a),len(user_b),len(df))
del user_b

user_a['user_encode'] = 1
user_a['user_encode'] = user_a['user_encode'].cumsum()
user_a, = dask.persist(user_a)
#_ = wait(user_a)

4434442 7708358 20872940
10806243 7708358 20872940
CPU times: user 4.14 s, sys: 1.84 s, total: 5.98 s
Wall time: 5.97 s


In [31]:
%%time
df = df.merge(user_a,left_on='a_user_id',right_on='user_id',how='left')
df = df.drop(['a_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'a_user_id' for i in df.columns]
df, = dask.persist(df)
#_ = wait(df)

CPU times: user 1.97 s, sys: 1.21 s, total: 3.18 s
Wall time: 3.19 s


In [32]:
%%time
df = df.merge(user_a,left_on='b_user_id',right_on='user_id',how='left')
df = df.drop(['b_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'b_user_id' for i in df.columns]
df, = dask.persist(df)
#wait(df)
del user_a
df.head()

CPU times: user 2.01 s, sys: 1.1 s, total: 3.1 s
Wall time: 3.11 s


,hashtags,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,...,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id
0,5F5E25FDD0B767C4BB6DE6473DC6AE33,<NA>,<NA>,1582018502,3388192,0,True,1312003421,4,206,...,0,0,0,13801804,54,2,8,1302828,553866,228624
1,E6D1192E7880DEE2E18C3D4489970918,<NA>,<NA>,1581798652,916,891,False,1496942442,504,938,...,0,0,0,8752519,54,2,4,1056460,430744,356802
2,<NA>,D1749EC58B796AA39D94DBE4153C1F62,7471C2E3375040BA50F8044E5D52B083,1581764307,171597,423,True,1462544521,76,69,...,0,0,0,8180770,54,2,12,1495267,645271,273905
3,3E73FEBD954B9E5A5C85F2AFFD5ED49D\tA62335257DDD...,<NA>,<NA>,1581869312,3190,24,False,1510154461,632,18,...,0,0,0,3728019,31,1,12,1321218,452570,550597
4,<NA>,<NA>,<NA>,1581904827,1760209,1,False,1416406065,255,113,...,0,0,0,11920974,47,2,4,986636,647160,122918


In [33]:
%%time
df = df.repartition(npartitions=NP)
df, = dask.persist(df)
#_ = wait(df)

CPU times: user 76 ms, sys: 108 ms, total: 184 ms
Wall time: 180 ms


In [34]:
%%time
df.to_parquet('./dask_input/step1_output',write_index=False)

CPU times: user 936 ms, sys: 2.13 s, total: 3.07 s
Wall time: 1.07 s


In [35]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 1.0 minutes
